In [ ]:
# recbole GRU4Rec 모델 base test

In [ ]:
# 1.라이브러리 임포트
import logging
from logging import getLogger
from recbole.quick_start import run_recbole
from recbole.quick_start.quick_start import load_data_and_model
from recbole.trainer import Trainer
from recbole.utils.case_study import full_sort_scores
from recbole.utils.case_study import full_sort_topk
from recbole.utils import init_seed, init_logger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
import pandas as pd
import gdown
import gzip
import json

In [ ]:
# 2.데이터 다운로드 & 리스트로 읽기

# 지난 Sequential Recommendation 실습에서 사용했던 데이터 사용
file_id = "1JO1Y3McBAPQuXHG1tezbk1n7_MnegA_1"
output = "./goodreads_reviews_spoiler.json.gz" # 저장 위치 및 저장할 파일 이름
gdown.download(id=file_id, output=output, quiet=False)

def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
    return data

review_data = load_data(output) # 약 1분 가량 소요됨

In [ ]:
# 3. 데이터프레임으로 읽기

df = pd.DataFrame(review_data)
df.head()
#df.info()

In [ ]:
# 4-1. 전처리 1

# user_id, book_id, timestamp만 남기고 간단히 전처리
# 커스텀 데이터 생성 테스트 용도 이므로 10%만 샘플링해서 처리함
df = df.sample(frac=0.1, random_state=42)[["user_id", "book_id", "timestamp"]].reset_index(drop=True)
df.head()

In [ ]:
# 4-2. 전처리 2

# 데이터 타입을 숫자형으로 전처리
df['timestamp'] = pd.to_datetime(df['timestamp']).astype(int)

# 사용자 아이디/아이템 아이디 인덱스로 변환

unique_users = df["user_id"].unique()
unique_items = df["book_id"].unique()

user_id2idx = {v: k for k, v in enumerate(unique_users)}
item_id2idx = {v: k for k, v in enumerate(unique_items)}

df["user_idx"] = df["user_id"].map(user_id2idx)
df["item_idx"] = df["book_id"].map(item_id2idx)

df.head()

In [ ]:
# 4.3 전처리 3

# RecBole 형식으로 컬럼명 변경
df = df.sort_values(["user_idx", "item_idx", "timestamp"])
df = df.rename(columns={"user_idx": "user_idx:token", "item_idx": "item_idx:token", "timestamp":"timestamp:float"})

df.head()

In [ ]:
# 4.4 전처리 4

# 전처리 데이터 저장
!mkdir ./recbole_data
df[["user_idx:token", "item_idx:token", "timestamp:float"]].to_csv("./recbole_data/recbole_data.inter", sep='\t', index=False)

In [ ]:
pd.read_csv("./recbole_data/recbole_data.inter", sep="\t")

In [ ]:
# 5. config 설정

parameter_dict = {
    # 데이터 설정
    'data_path': './',
    'USER_ID_FIELD': 'user_idx',
    'ITEM_ID_FIELD': 'item_idx',
    'TIME_FIELD': 'timestamp',
    
    # 필터링 설정
    'user_inter_num_interval': '[2,inf)',
    'item_inter_num_interval': '[2,inf)',
    'load_col': {'inter': ['user_idx', 'item_idx', 'timestamp']},
    
    'train_neg_sample_args': None,
    'epochs': 5,
    'stopping_step': 3,
    
    'eval_batch_size': 1024,
    'MAX_ITEM_LIST_LENGTH': 50,
    
    # 평가 설정
    'eval_args': {
        'split': {'RS': [0.9, 0.1, 0.0]},  # 90% 학습, 10% 검증
        'group_by': 'user',                # 사용자별로 평가
        'order': 'TO',                     # 시간 순서 유지
        'mode': 'full'                     # 모든 아이템 대상 평가
    },
    
    'device': 'cuda'
}

config = Config(model='GRU4Rec', dataset='recbole_data', config_dict=parameter_dict)   # recbole_data  디렉토리명


""" 
data_path: 데이터 파일이 있는 디렉토리.
USER_ID_FIELD: 사용자 ID 컬럼명 지정.
user_inter_num_interval: 최소 5번 이상 상호작용한 사용자만 사용.
item_inter_num_interval: 최소 5번 이상 나타난 아이템만 사용.
train_neg_sample_args: 학습 시 "음성 샘플(negative sample)"을 따로 뽑지 않겠다는 설정. 
                       추천시스템에서 모델이 "좋아한 아이템(positive)"과 "안 좋아한 아이템(negative)"을 구분해서 학습할 때, 
                       보통은 안 좋아한 아이템을 무작위로 뽑아(negative sampling) 같이 학습해. 
                       하지만, 어떤 모델(예: CrossEntropy loss 기반 모델)에서는 이런 샘플링이 필요 없어서 None으로 설정해.
epochs: 학습 반복 횟수.
stopping_step: 검증(validation) 성능이 3 번 연속으로 좋아지지 않으면 학습을 멈춤
MAX_ITEM_LIST_LENGTH: 사용자당 최대 아이템 시퀀스 길이.
split: 데이터 분할 비율 (학습:검증:테스트).
group_by: 평가 단위. 'user'는 사용자별로 평가.
order: 'TO'는 시간 순서(Time Order) 유지.
mode: 'full'은 모든 아이템을 후보로 평가.


1. split: 데이터 분할 방식
{'RS': [0.9, 0.1, 0.0]}

RS는 Ratio-based Splitting(비율 기반 분할)을 의미해.

[0.9, 0.1, 0.0]은 전체 데이터를 **90%는 학습(train), 10%는 검증(valid), 0%는 테스트(test)**로 나눈다는 뜻이야.

즉, 모델을 학습할 때 90% 데이터를 사용하고, 10%로 성능을 검증해.

2. group_by: 사용자별 평가
'user'

데이터를 사용자 단위로 그룹화해서 평가한다는 뜻이야.

즉, 각 사용자의 행동 기록을 따로 분리해서, 사용자별로 모델 성능을 측정해.

추천시스템에서는 보통 사용자별로 평가하는 게 일반적이야.

3. order: 시간 순서 유지
'TO' (Temporal Ordering)

데이터를 시간 순서대로 정렬해서 분할한다는 뜻이야.

예를 들어, 한 사용자가 여러 아이템을 순서대로 소비했다면, 그 순서를 그대로 유지해서 학습/검증 데이터로 나눠.

현실적인 추천 환경(미래 예측)에 더 가까운 방식이야.

4. mode: 전체 아이템 대상 평가
'full'

모델이 모든 아이템 후보 중에서 추천 리스트를 생성한다는 뜻이야.

즉, 평가할 때 "정답 아이템"과 함께 전체 아이템을 대상으로 랭킹을 매겨서 성능을 측정해.

샘플링된 일부 아이템만으로 평가하는 방식(uniN, popN)과 달리, 더 엄격하고 현실적인 평가야. 

"""

In [ ]:
# 6. 시드 & 로그 설정

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

#logger.info(config)

In [ ]:
# 7. 데이타셋 만들기

dataset = create_dataset(config)
#logger.info(dataset)

train_data, valid_data, test_data = data_preparation(config, dataset)

""" for data in train_data:
    print(data)
    break """

In [ ]:
# 8. 모델/ 트레이너 생성및 훈련

model = GRU4Rec(config, train_data.dataset).to(config['device'])
trainer = Trainer(config, model)
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

print(best_valid_score, best_valid_result)

In [ ]:
# 9. 메모리 정리

import gc
gc.collect()